# ViewSet

## View

요청을 적절하게 해석하고, Response 를 리턴하기 위해 논리적인 로직을 구현하는 곳

## ViewSet

비슷한 구조를 가진 뷰를 한 곳에서 사용할 수 있도록 해주는 추상 클래스

예를 들어 아래와 같은 API 를 만들어야 한다면 `ViewSet` 하나를 사용하여 여러 `View` 를 사용하지 않고 작성할 수 있습니다.

```
GET /api/users/
POST /api/users/
GET /api/users/<int:id>/
POST /api/users/<int:id>/set_staff
```

### 특징 ( View 와 다른 점 )

- **반복된 논리를 중복하지 않고 일관된 형태로 작성이 가능**
- 일반적인 패턴을 추상화했기 때문에 빠르게 API 작성 가능
- HTTP Method 를 사용하지 않음 ( get -> list, post -> create )
- CRDU 이외의 동작을 담을 수 있음
- URL 연결을 자동화할 수 있음

**지금은 특징들을 잘 모르더라도**

**문서를 보고 다른 것과 어떻게 다른지,**

**다른 방법으로 구현해보며 각각의 특징들을 익혀갑니다.**

---

# ViewSet 사용해보기


https://www.django-rest-framework.org/api-guide/viewsets/#example

https://www.django-rest-framework.org/api-guide/viewsets/#viewset-actions


## 목적
- 기본적인 ViewSet 을 사용하는 방법을 이해합니다.
- ViewSet 에서 HTTP Method 가 어떤 메서드에 바인딩되는지 이해합니다.

---

## 문제

위 링크를 을 참고하여

1. 생성(`POST`), 수정(`PATCH`), 삭제(`DELETE`) API 를 사용가능하도록 하게 해주세요.


### 조건
- `viewsets.ViewSet` 을 상속받아 View 를 작성해주세요.

In [1]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"

        
# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [2]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    allow_methods = dec(b'eyJwb3N0IjogWyJjcmVhdGUiXSwgInBhdGNoIjogWyJwYXJ0aWFsX3VwZGF0ZSJdLCAiZGVsZXRlIjogWyJkZXN0cm95Il19')
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, _ = get_actions_from_router(router)
    result_methods = allow_methods.copy()
    
    for method in group_by_methods:
        if method in result_methods:
            del result_methods[method]

    assert not result_methods, f'{", ".join([value for key, values in result_methods.items() for value in values])} 이(가) 구현되지 않았습니다.'
    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


---

# Action 확장해보기


https://www.django-rest-framework.org/api-guide/viewsets/#marking-extra-actions-for-routing


## 목적
- ViewSet 에서 action 을 확장하는 법을 이해합니다.

---

## 문제

위 링크 을 참고하여

`POST /users/<int:pk>/set_staff` 형태가 되도록 ViewSet 를 작성해주세요.

```
POST /users/<int:pk>/set_staff

STATUS 200
BODY
```

1. 이 API 는 `<int:pk>` 유저의 `is_staff` 속성을 True 로 바꾸는 동작을 합니다.
2. Response 는 status_code = 200 만 내려주면 됩니다.


### 조건

- `viewsets.ViewSet` 을 상속받아 View 를 작성해주세요.
- `pk` 에 맞는 유저가 존재하지 않을 시 않을 시 `404` 응답을 해주어야합니다. ( `rest_framework.generics.get_object_or_404` 를 사용하거나 참고해주세요. )

In [5]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"

        
# 이 부분을 완성해주세요!
# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [4]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    action_name = dec(b'InNldF9zdGFmZiI=')
    allow_method = dec(b'InBvc3Qi')
    
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    _, group_by_actions = get_actions_from_router(router)

    assert group_by_actions.get(action_name, None), 'set_staff action 이 구현되지 않았습니다.'
    assert allow_method in group_by_actions.get(action_name, []), f'set_staff action 는 POST Method 를 허용해야합니다. 현재 set_staff action 은 {", ".join(group_by_actions.get(action_name, []))} 을 허용중입니다.'
    
    factory = APIRequestFactory()
    view = UserViewSet.as_view(actions={allow_method: action_name})
    user = baker.make(User, is_staff=False)
    response = view(factory.post(''), pk=user.pk)

    assert response.status_code == 200, f'{allow_method} API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    user.refresh_from_db()
    assert user.is_staff, f'{allow_method} API 호출 결과 유저의 is_staff 속성이 True 로 설정되지 않았습니다.'
    
    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


---

# 하나의 Custom Action URL 에 여러 HTTP Method 허용하기


https://www.django-rest-framework.org/api-guide/viewsets/#routing-additional-http-methods-for-extra-actions


## 목적
- ViewSet 에서 하나의 Custom Action 을 가지고 여러가지 HTTP Method 를 붙이는 방법을 이해합니다.

### 조건

- `viewsets.ViewSet` 을 상속받아 View 를 작성해주세요.
- `pk` 에 맞는 유저가 존재하지 않을 시 않을 시 `404` 응답을 해주어야합니다. ( `rest_framework.generics.get_object_or_404` 를 사용하거나 참고해주세요. )

---

## 문제 1

아래 두 API 를 구현해야합니다.
우선 문서를 보지 않고 하나의 Custom Action 을 만들어 GET, DELETE 두 동작이 정상 작동하도록 View 를 작성해주세요.

- `GET /users/<int:pk>/email` : 유저의 이메일만 가져오는 API

```
GET /users/<int:pk>/email

STATUS 200
BODY

{
    'admin@ashe.kr'
}
```


- `DELETE /users/<int:pk>/email` : 유저의 이메일을 삭제하는 API

```
DELETE /users/<int:pk>/email

STATUS 204
BODY

```

In [6]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"

        
# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [7]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    action_name = dec(b'ImVtYWlsIg==')
    allow_method = dec(b'WyJnZXQiLCAiZGVsZXRlIl0=')
    
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, group_by_actions = get_actions_from_router(router)

    assert group_by_actions.get(action_name, None), 'email action 이 구현되지 않았습니다.'
    assert allow_method == group_by_actions.get(action_name, []), f'email action 은 GET, DELETE Method 를 허용해야합니다. 현재 email action 은 {", ".join(group_by_actions.get(action_name, []))} 을 허용중입니다.'

    factory = APIRequestFactory()

    view = UserViewSet.as_view(actions={'get': action_name})
    user = baker.make(User, email='admin@ashe.kr')
    response = view(factory.get(''), pk=user.pk)

    assert response.status_code == 200, f'get API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert response.data == 'admin@ashe.kr', f'get API 호출 결과는 {"admin@ashe.kr"} 이 되어야 합니다: 현재: {response.data}'
    
    view = UserViewSet.as_view(actions={'delete': action_name})
    response = view(factory.delete(''), pk=user.pk)
    
    user.refresh_from_db()
    assert response.status_code == 204, f'delete API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert not user.email, 'delete API 호출 결과 유저의 email 속성이 지워지지 않았습니다.'

    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


## 문제 2

문서를 읽고나서 두개의 Custom Action(email, delete_email) 을 만들어 GET, DELETE 두 동작이 정상 작동하도록 View 를 작성해주세요.

- `GET /users/<int:pk>/email` : 유저의 이메일만 가져오는 API

```
GET /users/<int:pk>/email

STATUS 200
BODY

{
    'admin@ashe.kr'
}
```


- `DELETE /users/<int:pk>/email` : 유저의 이메일을 삭제하는 API

```
DELETE /users/<int:pk>/email

STATUS 204
BODY

```

In [8]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"

        
# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [9]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, group_by_actions = get_actions_from_router(router)

    action_name = 'email'
    allow_method = ['get']
    assert group_by_actions.get(action_name, None), 'email action 이 구현되지 않았습니다.'
    assert allow_method == group_by_actions.get(action_name, []), f'email action 은 GET Method 를 허용해야합니다. 현재 email action 은 {", ".join(group_by_actions.get(action_name, []))} 을 허용중입니다.'
    
    factory = APIRequestFactory()
    user = baker.make(User, email='admin@ashe.kr')

    view = UserViewSet.as_view(actions={'get': 'email'})
    response = view(factory.get(''), pk=user.pk)

    assert response.status_code == 200, f'get API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert response.data == 'admin@ashe.kr', f'get API 호출 결과는 {"admin@ashe.kr"} 이 되어야 합니다: 현재: {response.data}'
    
    action_name = 'delete_email'
    allow_method = ['delete']
    assert group_by_actions.get(action_name, None), 'delete_email action 이 구현되지 않았습니다.'
    assert allow_method == group_by_actions.get(action_name, []), f'delete_email action 은 DELETE Method 를 허용해야합니다. 현재 email action 은 {", ".join(group_by_actions.get(action_name, []))} 을 허용중입니다.'
    
    view = UserViewSet.as_view(actions={'delete': 'delete_email'})
    response = view(factory.delete(''), pk=user.pk)
    
    user.refresh_from_db()
    assert response.status_code == 204, f'delete API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert not user.email, 'delete API 호출 결과 유저의 email 속성이 지워지지 않았습니다.'

    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


---

# 다양한 ViewSet 사용해보기

- https://www.django-rest-framework.org/api-guide/viewsets/#modelviewset

- https://www.django-rest-framework.org/api-guide/viewsets/#custom-viewset-base-classes


## 목적

- ViewSet 을 상속하는 다양한 ViewSet 들에 대해 이해하고 넘어갑니다.

---

## 문제 1

위 링크를 을 참고하여

1. GenericViewSet 하나만 상속받고 `get_queryset`, `get_object` 를 사용하여 목록(GET), 상세(GET), 삭제(DELETE) API 를 사용가능하도록 하게 해주세요.

In [12]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"


# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [11]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory
from rest_framework.utils.serializer_helpers import ReturnList

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    allow_methods = {'get': ['list', 'retrieve'], 'delete': ['destroy']}
    
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, _ = get_actions_from_router(router)
    result_methods = allow_methods.copy()
    
    for method in group_by_methods:
        if method in result_methods:
            del result_methods[method]

    assert not result_methods, f'{", ".join([value for key, values in result_methods.items() for value in values])} 이(가) 구현되지 않았습니다.'
            
    # Method 체크
    
    factory = APIRequestFactory()
    user = baker.make(User, email='admin@ashe.kr')
    
    # List
    view = UserViewSet.as_view(actions={'get': 'list'})
    response = view(factory.get(''))

    assert response.status_code == 200, f'list API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert type(response.data) == ReturnList, f'list API 의 호출결과가 list 타입이 아닙니다: 현재 {type(response.data)}'
    assert {data.get('email'): data for data in response.data}.get('admin@ashe.kr'), f'list API 의 호출 결과에서 테스트 데이터를 찾을 수 없습니다.'
    
    # Retrieve
    view = UserViewSet.as_view(actions={'get': 'retrieve'})
    response = view(factory.get(''), pk=user.pk)

    assert response.status_code == 200, f'retrieve API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    
    # Delete
    view = UserViewSet.as_view(actions={'delete': 'destroy'})
    response = view(factory.delete(''), pk=user.pk)

    assert response.status_code == 204, f'destroy API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert not User.objects.filter(pk=user.pk).first(), f'destroy API 호출 결과로 User 가 삭제되지 않았습니다.'

    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


## 문제 2

위 링크를 을 참고하여

2. ModelViewSet 을 상속받아 목록(GET), 생성(POST), 상세(GET), 수정(PUT), 수정(PATCH), 삭제(DELETE) API 를 사용가능하도록 하게 해주세요.

In [13]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"


# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [14]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory
from rest_framework.utils.serializer_helpers import ReturnList

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    allow_methods = dec(b'eyJnZXQiOiBbImxpc3QiLCAicmV0cmlldmUiXSwgInBvc3QiOiBbImNyZWF0ZSJdLCAicHV0IjogWyJ1cGRhdGUiXSwgInBhdGNoIjogWyJwYXJ0aWFsX3VwZGF0ZSJdLCAiZGVsZXRlIjogWyJkZXN0cm95Il19')
    
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, _ = get_actions_from_router(router)
    result_methods = allow_methods.copy()
    
    for method in group_by_methods:
        if method in result_methods:
            del result_methods[method]

    assert not result_methods, f'{", ".join([value for key, values in result_methods.items() for value in values])} 이(가) 구현되지 않았습니다.'
            

    # Method 체크
    
    factory = APIRequestFactory()
    user = baker.make(User, email='admin@ashe.kr')
    
    # List
    view = UserViewSet.as_view(actions={'get': 'list'})
    response = view(factory.get(''))

    assert response.status_code == 200, f'list API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert type(response.data) == ReturnList, f'list API 의 호출결과가 list 타입이 아닙니다: 현재 {type(response.data)}'
    assert {data.get('email'): data for data in response.data}.get('admin@ashe.kr'), f'list API 의 호출 결과에서 테스트 데이터를 찾을 수 없습니다.'
    
    # Retrieve
    view = UserViewSet.as_view(actions={'get': 'retrieve'})
    response = view(factory.get(''), pk=user.pk)

    assert response.status_code == 200, f'retrieve API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    
    # Create
    view = UserViewSet.as_view(actions={'post': 'create'})
    request_data = UserSerializer(baker.prepare(User, email='second@ashe.kr')).data
    response = view(factory.post('', request_data, format='json'))

    assert response.status_code == 201, f'create API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert User.objects.filter(email='second@ashe.kr').first(), 'create API 호출 결과로 User 가 생성되지 않았습니다.'
    
    # Update
    view = UserViewSet.as_view(actions={'put': 'update'})
    user.refresh_from_db()
    request_data = UserSerializer(user).data
    request_data['email'] = 'third@ashe.kr'
    response = view(factory.put('', request_data, format='json'), pk=user.pk)

    assert response.status_code == 200, f'update API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    user.refresh_from_db()
    assert user.email == 'third@ashe.kr', 'update API 호출 결과로 email 이 업데이트 되지 않았습니다.'
    
    # Partial-Update
    view = UserViewSet.as_view(actions={'patch': 'partial_update'})
    user.refresh_from_db()
    request_data = UserSerializer(user).data
    request_data['email'] = 'forth@ashe.kr'
    response = view(factory.patch('', request_data, format='json'), pk=user.pk)

    assert response.status_code == 200, f'partial_update API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    user.refresh_from_db()
    assert user.email == 'forth@ashe.kr', 'partial_update API 호출 결과로 email 이 업데이트 되지 않았습니다.'
    
    # Delete
    view = UserViewSet.as_view(actions={'delete': 'destroy'})
    response = view(factory.delete(''), pk=user.pk)

    assert response.status_code == 204, f'destroy API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert not User.objects.filter(pk=user.pk).first(), f'destroy API 호출 결과로 User 가 삭제되지 않았습니다.'

    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


## 문제 3

3. GenericViewSet 을 상속받고 `mixins` 를 통해 목록(GET), 생성(CREATE) 만 가능하도록 하게 해주세요.

In [19]:
from django.contrib.auth import get_user_model
from rest_framework import viewsets, serializers, mixins
from rest_framework.response import Response
from rest_framework.decorators import action
from rest_framework.generics import get_object_or_404

User = get_user_model()


# 지금은 몰라도 됩니다!
class UserSerializer(serializers.ModelSerializer):
    class Meta:
        model = User
        fields = "__all__"


# 이 부분을 완성해주세요!
class UserViewSet:
    pass

In [18]:
# 잘 작성되었는지 검증하는 코드입니다.
from django.db import transaction
from rest_framework.routers import SimpleRouter
from rest_framework.test import APIRequestFactory
from rest_framework.utils.serializer_helpers import ReturnList

from model_bakery import baker

from utils import enc, dec, get_actions_from_router, AtomicRollback


with AtomicRollback():
    allow_methods = {'get': ['list', 'retrieve'], 'post': ['create']}
    
    router = SimpleRouter()
    router.register('users', UserViewSet)
    
    group_by_methods, _ = get_actions_from_router(router)
    result_methods = allow_methods.copy()
    
    for method in group_by_methods:
        if method in result_methods:
            del result_methods[method]

    assert not result_methods, f'{", ".join([value for key, values in result_methods.items() for value in values])} 이(가) 구현되지 않았습니다.'
            

    # Method 체크
    
    factory = APIRequestFactory()
    user = baker.make(User, email='admin@ashe.kr')
    
    # List
    view = UserViewSet.as_view(actions={'get': 'list'})
    response = view(factory.get(''))

    assert response.status_code == 200, f'list API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert type(response.data) == ReturnList, f'list API 의 호출결과가 list 타입이 아닙니다: 현재 {type(response.data)}'
    assert {data.get('email'): data for data in response.data}.get('admin@ashe.kr'), f'list API 의 호출 결과에서 테스트 데이터를 찾을 수 없습니다.'
    
    # Create
    view = UserViewSet.as_view(actions={'post': 'create'})
    request_data = UserSerializer(baker.prepare(User, email='second@ashe.kr')).data
    response = view(factory.post('', request_data, format='json'))

    assert response.status_code == 201, f'create API 호출 결과의 status_code 가 일치하지 않습니다: 현재: {response.status_code}'
    assert User.objects.filter(email='second@ashe.kr').first(), 'create API 호출 결과로 User 가 생성되지 않았습니다.'

    print('테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.')

테스트에 성공하였습니다! 다음 블럭으로 넘어가주세요.


# --- 끝 ---